# Semantic Chunking
- Instead of splitting text at arbitrary lengths, use Natural Language Processing (NLP) to split at meaningful boundaries:
- Sentence-based: Use NLP tools like Spacy or NLTK to split by sentences.
- Paragraph-based: Treat each paragraph as a chunk.
- Heading-based: Use document structure (e.g., titles, headings, subheadings) to define chunks.
- Slide-based: If the PDF is a slide deck, extract each slide as a chunk.

In [3]:
import nltk

# Download tokenizer model for Portuguese if not already downloaded
nltk.download("punkt")

from nltk.tokenize import sent_tokenize

def semantic_chunking(text, max_length=500):
    sentences = sent_tokenize(text, language="portuguese")  # Tokenize sentences in Portuguese
    chunks, current_chunk = [], ""

    for sent in sentences:
        if len(current_chunk) + len(sent) < max_length:
            current_chunk += " " + sent
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sent

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks


[nltk_data] Downloading package punkt to /home/ebezerra/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
import fitz  # PyMuPDF
def extract_text_from_pdf(pdf_path):
    """Extracts text from a PDF file."""
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text("text") for page in doc])
    return text

In [7]:
PDF_FILE = "../data/tokio_outubro_2024.pdf"  # Change this to your PDF file
print("📄 Extracting text from PDF...")
raw_text = extract_text_from_pdf(PDF_FILE)


📄 Extracting text from PDF...


In [8]:
raw_text

' \n1 \nTokio Marine Seguradora S.A – Cia 06190 \n \nCondomínio Processo SUSEP nº 15414.100909/2004-12 - Versão- Outubro/2024 \n \nSEGURO  \nTOKIO MARINE CONDOMÍNIO \nCondições Gerais  \nVersão 05 de Outubro de 2024 \n\n \n2 \nTokio Marine Seguradora S.A – Cia 06190 \n \nCondomínio Processo SUSEP nº 15414.100909/2004-12 - Versão- Outubro/2024 \n \nTOKIO MARINE CONDOMÍNIO- CONDIÇÕES GERAIS \nVersão 05 de Outubro/2024 \nProcesso SUSEP nº 15414.100909/2004-12 CONDOMÍNIO \nEste seguro é garantido pela Tokio Marine Seguradora S/A \nCNPJ 33.164.021/0001-00 \nVida em Grupo complementar ao Plano de Seguro Condomínio \nProcesso SUSEP nº 15414.004366/2006-67 e 15414.901913/2013-83 \n \n \nAPRESENTAÇÃO \n \nOlá, \nSeja bem-vindo (a)! \n \nAgradecemos a sua confiança em escolher a Tokio Marine. \n \nApresentamos de forma simples as Condições Gerais do seu seguro Condomínio, que estabelecem as normas de funcionamento \ndas coberturas contratadas. \n \nPara os devidos fins e efeitos, serão considera

In [13]:
# Define the output file path
OUTPUT_FILE = "../data/tokio_outubro_2024.txt"  

# Save extracted text to a text file
with open(OUTPUT_FILE, "w", encoding="utf-8") as file:
    file.write(raw_text)

In [14]:
import re

def remove_footer(text):
    """
    Removes footers matching the given pattern.
    """
    # Define a regex pattern based on your footer structure
    footer_pattern = r"\d+\s*\nTokio Marine Seguradora S\.A – Cia \d+\s*\nCondomínio Processo SUSEP nº .* - Versão- .*"

    # Remove matching footer patterns
    cleaned_text = re.sub(footer_pattern, "", text, flags=re.MULTILINE)

    return cleaned_text.strip()

# Load extracted text from file
input_file = "../data/tokio_outubro_2024.txt"
output_file = "../data/tokio_outubro_2024_cleaned.txt"

with open(input_file, "r", encoding="utf-8") as file:
    raw_text = file.read()

# Remove footers
cleaned_text = remove_footer(raw_text)

# Save cleaned text
with open(output_file, "w", encoding="utf-8") as file:
    file.write(cleaned_text)

print("✅ Footer removed successfully!")


✅ Footer removed successfully!


In [ ]:
# nltk.download('punkt_tab')

In [11]:
chunks = semantic_chunking(raw_text)

In [12]:
chunks

['',
 '1 \nTokio Marine Seguradora S.A – Cia 06190 \n \nCondomínio Processo SUSEP nº 15414.100909/2004-12 - Versão- Outubro/2024 \n \nSEGURO  \nTOKIO MARINE CONDOMÍNIO \nCondições Gerais  \nVersão 05 de Outubro de 2024 \n\n \n2 \nTokio Marine Seguradora S.A – Cia 06190 \n \nCondomínio Processo SUSEP nº 15414.100909/2004-12 - Versão- Outubro/2024 \n \nTOKIO MARINE CONDOMÍNIO- CONDIÇÕES GERAIS \nVersão 05 de Outubro/2024 \nProcesso SUSEP nº 15414.100909/2004-12 CONDOMÍNIO \nEste seguro é garantido pela Tokio Marine Seguradora S/A \nCNPJ 33.164.021/0001-00 \nVida em Grupo complementar ao Plano de Seguro Condomínio \nProcesso SUSEP nº 15414.004366/2006-67 e 15414.901913/2013-83 \n \n \nAPRESENTAÇÃO \n \nOlá, \nSeja bem-vindo (a)!',
 'Agradecemos a sua confiança em escolher a Tokio Marine. Apresentamos de forma simples as Condições Gerais do seu seguro Condomínio, que estabelecem as normas de funcionamento \ndas coberturas contratadas. Para os devidos fins e efeitos, serão consideradas some

# Using llmsherpa

In [ ]:
# Install package
# !pip install --upgrade --quiet llmsherpa

In [23]:
# get all documents under the folder
import os
import glob
from datetime import datetime
import time

file_location = "../data/apolices"

from llmsherpa.readers import LayoutPDFReader

llmsherpa_api_url = "http://localhost:5001/parseDocument?renderFormat=all"

pdf_files = glob.glob(file_location + '/*.pdf')

print(f'#PDF files found: {len(pdf_files)}!')
pdf_reader = LayoutPDFReader(llmsherpa_api_url)

# parse documents and create graph
startTime = datetime.now()

for pdf_file in pdf_files:
    print(pdf_file)
    doc = pdf_reader.read_pdf(pdf_file)

    # find the first / in pdf_file from right
    idx = pdf_file.rfind('/')
    pdf_file_name = pdf_file[idx+1:]

    # open a local file to write the JSON
    with open(pdf_file_name + '.json', 'w') as f:
        # convert doc.json from a list to string
        f.write(str(doc.json))

print(f'Total time: {datetime.now() - startTime}')

#PDF files found: 1!
../data/apolices/tokio_outubro_2024.pdf


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [32]:
from langchain_community.document_loaders.llmsherpa import LLMSherpaFileLoader

loader = LLMSherpaFileLoader(
    file_path="../data/apolices/tokio_outubro_2024.pdf",
    new_indent_parser=False,
    apply_ocr=False,
    strategy="sections",
    llmsherpa_api_url="http://localhost:5001/api/parseDocument?renderFormat=all",
)
docs = loader.load()

KeyError: 'return_dict'

In [26]:
docs

[]